In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from json import dumps
import pandas as pd
from kafka import KafkaProducer
from json import dumps
import time

ModuleNotFoundError: No module named 'kafka.vendor.six.moves'

In [1]:
import pandas as pd
happiness_2015 = pd.read_csv("../data/2015.csv")
happy_2016 = pd.read_csv("../data/2016.csv")
happier_2017 = pd.read_csv("../data/2017.csv")
happiest_2018 = pd.read_csv("../data/2018.csv")
happierness_2019 = pd.read_csv("../data/2019.csv")

In [2]:
from transform_happy import transformar_datasets

happy_df = transformar_datasets(happiness_2015, happy_2016, happier_2017, happiest_2018, happierness_2019)

happy_df.head()

c:\Users\JSLV3\Documents\5to Semestre\ETL\WORKSHOP 3\WORKSHOP3\main\transform_happy.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  happiest_2018.rename(columns=rename_dict_2018_2019, inplace=True)


,happiness_score,economy,social_support,life_expectancy,freedom,government_corruption,generosity,year,continent_Asia,continent_Asia/Europe,continent_Europe,continent_Europe/Asia,continent_North America,continent_Oceania,continent_South America
0,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015,0,0,1,0,0,0,0
1,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015,0,0,1,0,0,0,0
2,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015,0,0,1,0,0,0,0
3,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015,0,0,1,0,0,0,0
4,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015,0,0,0,0,1,0,0


In [5]:
X = happy_df.drop(columns=['happiness_score'])
y = happy_df['happiness_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [6]:
X_train_shape = X_train.shape
X_test_shape = X_test.shape
y_train_shape = y_train.shape
y_test_shape = y_test.shape

X_train_shape, X_test_shape, y_train_shape, y_test_shape

((546, 14), (235, 14), (546,), (235,))

In [7]:
X_test["happiness_score"] = y_test
happy_test = X_test

# Imprimir la forma del DataFrame combinado
print(f"Shape of happy_test: {happy_test.shape}")

# Mostrar las primeras filas del DataFrame combinado
happy_test.head()


Shape of happy_test: (235, 15)


,economy,social_support,life_expectancy,freedom,government_corruption,generosity,year,continent_Asia,continent_Asia/Europe,continent_Europe,continent_Europe/Asia,continent_North America,continent_Oceania,continent_South America,happiness_score
365,1.416915,1.436338,0.913476,0.505626,0.163761,0.120573,2017,1,0,0,0,0,0,0,5.920
766,0.274000,0.757000,0.505000,0.142000,0.078000,0.275000,2019,0,0,0,0,0,0,0,3.973
774,0.191000,0.560000,0.495000,0.443000,0.089000,0.218000,2019,0,0,0,0,0,0,0,3.410
361,0.786441,1.548969,0.498273,0.658249,0.246528,0.415984,2017,1,0,0,0,0,0,0,5.971
29,1.053510,1.248230,0.787230,0.449740,0.084840,0.114510,2015,0,0,0,0,0,0,1,6.574


In [ ]:
producer = KafkaProducer(
    value_serializer=lambda m: dumps(m).encode('utf-8'),
    bootstrap_servers=['localhost:9092']
)

# Función para enviar mensajes con manejo de errores
def send_message(producer, topic, message):
    try:
        producer.send(topic, value=message)
        print(f"Sent: {message}")
    except Exception as e:
        print(f"Error sending message: {e}")

# Enviar filas del DataFrame como mensajes
for _, row in happy_test.iterrows():
    message = row.to_dict()
    send_message(producer, 'happy_test', message)
    time.sleep(2)  # Controla la tasa de envío (puede ser ajustada o eliminada)

# Asegurar que todos los mensajes se envían antes de cerrar
producer.flush()
producer.close()
